# Question Answering System

In [2]:
# Import necessary libraries
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the SQuAD dataset
dataset = load_dataset("squad")

# Split the dataset into training and validation sets
train_data = dataset['train']
validation_data = dataset['validation']

print(f"Training set size: {len(train_data):,}")
print(f"Validation set size: {len(validation_data):,}\n")
print(f"Sample context:\n{train_data[0]['context']}")
print(f"Sample question:\n{train_data[0]['question']}")
print(f"Sample answer:\n{train_data[0]['answers']}")

Training set size: 87,599
Validation set size: 10,570

Sample context:
Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
Sample question:
To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Sample answer:
{'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}


## 1. Heuristic QA System 🛠️
Understand the basic heuristic approach for simple question answering, including text similarity and search techniques.

**Approach:** Given a question and a context, the system will split the context into sentences and measure the similarity between the question and each sentence. The sentence with the highest similarity score will be returned as the answer.

1. **Context Extraction:** Extract relevant passages from the context using text similarity techniques.
2. **Text Similarity:** Measure similarity between the question and sentences in the context.
3. **Answer Extraction:** Identify the span of text that most likely contains the answer.

In [3]:
def find_answer_heuristic(context, question):
    """
    Find the answer to a question in a context using a heuristic approach.
    """
    # Tokenize context into sentences
    sentences = context.split('.') # splits the context into sentences not words
    
    # Vectorize the question and sentences
    vectorizer = TfidfVectorizer().fit(sentences + [question])
    question_vec = vectorizer.transform([question])
    sentences_vec = vectorizer.transform(sentences)
    
    # Calculate cosine similarity between question and each sentence
    similarities = cosine_similarity(question_vec, sentences_vec).flatten()
    
    # Find the most similar sentence
    most_similar_idx = similarities.argmax()
    most_similar_sentence = sentences[most_similar_idx]
    
    return most_similar_sentence.strip()

# Test the heuristic approach on a sample question
sample_context = train_data[0]['context']
sample_question = train_data[0]['question']
sample_answer = train_data[0]['answers']['text'][0]

predicted_answer = find_answer_heuristic(sample_context, sample_question)

print(f"Context: {sample_context}")
print(f"Question: {sample_question}")
print(f"Predicted Answer: {predicted_answer}")
print(f"Actual Answer: {sample_answer}")

Context: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Predicted Answer: It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858
Actual Answer: Saint Bernadette Soubirous


## 2. Model Evaluation 📊
Evaluate the effectiveness of the heuristic QA system on the validation set.

In [4]:
def evaluate_qa_system(validation_data, num_samples=100):
    """
    Evaluate the QA system on the validation data.
    """
    correct = 0
    for i in range(num_samples):
        context = validation_data[i]['context']
        question = validation_data[i]['question']
        actual_answer = validation_data[i]['answers']['text'][0]
        
        predicted_answer = find_answer_heuristic(context, question)
        
        # Check if the actual answer is contained within the predicted answer
        if actual_answer.lower() in predicted_answer.lower():
            correct += 1
    
    accuracy = correct / num_samples
    print(f"Accuracy on {num_samples} samples: {accuracy * 100:.2f}%")

# Evaluate the system
evaluate_qa_system(validation_data)

Accuracy on 100 samples: 64.00%


## 3. Test the QA System 🧪

In [16]:
sample_context = "My name is John. I live in Cairo, Egypt. I work as a software engineer."
sample_question = "Where do I live?"

predicted_answer = find_answer_heuristic(sample_context, sample_question)

print(f"Context: {sample_context}\n")
print(f"Question: {sample_question}")
print(f"- Predicted Answer: {predicted_answer}")

Context: My name is John. I live in Cairo, Egypt. I work as a software engineer.

Question: Where do I live?
- Predicted Answer: I live in Cairo, Egypt


## 4. Model Analysis and Improvements 🔍
Analyze the performance of the heuristic QA system and suggest potential improvements.

In [6]:
# Analyze a few examples where the model fails
for i in range(5):
    context = validation_data[i]['context']
    question = validation_data[i]['question']
    actual_answer = validation_data[i]['answers']['text'][0]
    
    predicted_answer = find_answer_heuristic(context, question)
    
    print(f"\nContext: {context}")
    print(f"Question: {question}")
    print(f"Predicted Answer: {predicted_answer}")
    print(f"Actual Answer: {actual_answer}")
    
    if actual_answer.lower() not in predicted_answer.lower(): # Wrong answer (model failed)
        print("#Prediction failed for this example.#")


Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
Question: Which NFL team represented the AFC at Super Bowl 50?
Predicted Answer: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season
Actua

## Conclusion 🎯
In this notebook, we developed a simple Question Answering (QA) system using a heuristic approach based on text similarity. The system was tested on the SQuAD dataset and provided answers by identifying the most relevant sentences in the context.

**Next steps** could involve exploring more advanced QA systems based on deep learning models like [BERT](../../../3.%20Advanced/bert/) or fine-tuning pre-trained models for QA tasks.